# Akka.Streams and Backoff
Part of the raison d'être for Akka.Streams in the first place is to faciliate support for [Reactive Streams](https://www.reactive-streams.org/) - the ability to ensure that in asynchronous producer-consumer systems a faster upstream producer can't overwhelm a slower downstream consumer.

Let's consider an example.

In [ ]:
#r "nuget: Akka.Streams, 1.4.24"

using System.Linq;
using System.Collections.Immutable;
using Akka;
using Akka.Actor;
using Akka.Streams;
using Akka.Streams.Dsl;

ActorSystem actorSystem = ActorSystem.Create("StreamsExample2");
IMaterializer materializer = actorSystem.Materializer();

// Fast upstream producer - instantly produces 10,000 requests
Source<int,NotUsed> source = Source.From(Enumerable.Range(0, 10000));

// Group into batches of 100
Source<IEnumerable<int>,NotUsed> batchingSource = source.Via(Flow.Create<int>().Grouped(100)); 

// output
var (task, sink) = Sink.Seq<(int, long)>()
    .PreMaterialize(materializer);
//.WithAttributes(new Attributes(new ActorAttributes.Dispatcher("akka.actor.synchronized-dispatcher")));

// simulate a slower downstream consumer - can only process 10 events per second
var slowSink = Flow.Create<IEnumerable<int>>().Delay(TimeSpan.FromMilliseconds(100))
    .Select(x => (x.Sum(), DateTime.UtcNow.Ticks)) // sum each group
    .To(sink);

batchingSource.RunWith(slowSink, materializer);

var output = await task;

foreach(var i in output){
    Console.WriteLine($"{i}");
}


Installed Packages Akka.Streams, 1.4.24

(4950, 637654561536058261)
(14950, 637654561536077747)
(24950, 637654561536078125)
(34950, 637654561536078145)
(44950, 637654561536078164)
(54950, 637654561536078184)
(64950, 637654561536078202)
(74950, 637654561536078220)
(84950, 637654561536078238)
(94950, 637654561536078260)
(104950, 637654561536078278)
(114950, 637654561536078295)
(124950, 637654561536078313)
(134950, 637654561536078330)
(144950, 637654561536078348)
(994950, 637654561536078394)


In this instance, we are going to simulate a slower downstream consumer by using [a `Delay<T>` flow](https://getakka.net/api/Akka.Streams.Dsl.DelayFlow-1.html), which creates 100ms pauses in the stream:

```csharp
Flow.Create<IEnumerable<int>>().Delay(TimeSpan.FromMilliseconds(100))
```

The upstream producer, our `Source<int>`, on the other hand, will immediately create 10,000 items that can be processed immediately:

```csharp
Source<int,NotUsed> source = Source.From(Enumerable.Range(0, 10000));
```

So what do we do? In our case, we need to equip our Akka.Streams graph with the ability to handle backpressure somehow - otherwise, this stream will take 1,000 seconds to run through to completion.

The approach we used in the sample above to do this is [a `Grouped<T,IEnumerable<T>>` flow](https://getakka.net/api/Akka.Streams.Dsl.FlowOperations.html#Akka_Streams_Dsl_FlowOperations_Grouped__3_Akka_Streams_Dsl_Flow___0___1___2__System_Int32_) - which will chunk upstream elements into groups of up to 100. When the `Delay<T>` backpressures, however, the grouping will stop and the original `Source<int>` will stop being read from until the downstream is ready to process again.

That's why you can see this type of pattern appear in the output:

```
Sum,   DateTime.UtcNow.Ticks
(4950, 637654561536058261)
(14950, 637654561536077747)
(24950, 637654561536078125)
```

The sums are growing larger because we're selecting larger integers in the sequence - and the tickcount is growing larger because the processing delay in computing the sum is being observed in real-time as the stream is processed.

This is an example of backpressure at work inside an Akka.NET Stream!